In [1]:
print("hello")

hello


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import os
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

res = llm.invoke("Hello, how are you?")

res.content

"Hello! I'm doing great, thank you for asking.\n\nAs an AI, I'm always ready to help. How can I assist you today?"

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("What are embeddings?")
len(vector)

3072

In [13]:
from langchain_community.document_loaders import PyPDFLoader

paths = ["C:\\Users\\SRJ\\SRJ\\Work\\agentic_ai\\Langgraph\\data\\Arihant GK 2025 - Himexam.pdf",  "C:\\Users\\SRJ\\SRJ\\Work\\agentic_ai\\Langgraph\\data\\modern_indian_history_tutorial.pdf"]
docs = []
for path in paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

In [15]:
len(docs)

451

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=25)
doc_splits = text_splitter.split_documents(docs)


In [18]:
len(doc_splits)

11766

In [20]:
from langchain_community.vectorstores import Chroma

vectorstore=Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chrome",
    embedding=embeddings
)

In [21]:
retriever=vectorstore.as_retriever()
retriever.invoke("what is the capital of France?")

[Document(metadata={'creationdate': '2024-09-08T14:34:17+05:30', 'creator': 'Adobe Acrobat Pro (64-bit) 23.1.20064', 'moddate': '2025-03-30T09:49:03+02:00', 'page': 136, 'page_label': '137', 'producer': '3.0.24 (5.1.10)', 'source': 'C:\\Users\\SRJ\\SRJ\\Work\\agentic_ai\\Langgraph\\data\\Arihant GK 2025 - Himexam.pdf', 'title': '', 'total_pages': 160}, page_content='Denmark Copenhagen Krone\nEgypt Cairo Egyptian Pound\nFrance Paris Franc, Euro\nGermany Berlin Euro'),
 Document(metadata={'creationdate': '2024-09-08T14:34:17+05:30', 'creator': 'Adobe Acrobat Pro (64-bit) 23.1.20064', 'moddate': '2025-03-30T09:49:03+02:00', 'page': 136, 'page_label': '137', 'producer': '3.0.24 (5.1.10)', 'source': 'C:\\Users\\SRJ\\SRJ\\Work\\agentic_ai\\Langgraph\\data\\Arihant GK 2025 - Himexam.pdf', 'title': '', 'total_pages': 160}, page_content='Countries with Capitals and Currencies\nCountry Capital Currency\nAfghanistan Kabul Afghani'),
 Document(metadata={'creationdate': '2024-09-08T14:34:17+05:30',

In [22]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(
    retriever,
    "retriever_GK",
    "Search and return information about General Knowledge and Indian History.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to General Knowledge and Indian History. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.",
    )

In [23]:
tools=[retriever_tool]

from langgraph.prebuilt import ToolNode

retriever_node=ToolNode(tools)

retriever_node

tools(tags=None, recurse=True, explode_args=False, func_accepts={'config': ('N/A', <class 'inspect._empty'>), 'store': ('store', None)}, tools_by_name={'retriever_GK': Tool(name='retriever_GK', description="Search and return information about General Knowledge and Indian History.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to General Knowledge and Indian History. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029033C69440>, retriever=VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000029033986510>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial